In [0]:
########## This code is for finding out the subject object from the heading


import spacy
subject_tokens=[]
object_tokens=[]
nlp = spacy.load("en_core_web_sm")
sentence_main="graviola shows promise in cancer cures and arthritis."
doc = nlp(sentence_main)
for chunk in doc.noun_chunks:
    #print(chunk.text, chunk.root.text, chunk.root.dep_,chunk.root.head.text)
    print(chunk.root.text, chunk.root.dep_)
    if (chunk.root.dep_=="nsubj"):
      subject_tokens.append(chunk.root.text)
      print(subject_tokens)
    elif (chunk.root.dep_=="dobj" or chunk.root.dep_=="pobj"):
      object_tokens.append(chunk.text)
      print(object_tokens)



graviola nsubj
['graviola']
cures pobj
['cancer cures']
arthritis conj


In [0]:
####### This code is for finding those sentences which contain the subject and the object


import re
import pandas as pd
line=[]
can_rel=[]
df=pd.read_excel(r'/content/pandas_simple.xlsx')
df['Body'] = df['Body'].str.lower()
for i in range(len(df.Body)):
    #if(i<2000):
    line.append(df.Body[i])
#print(line)
for sent in line:
    match = re.findall(".*graviola.*cancer.*", sent)   ##### returns back sentences which has the word that followed by subject and object
    #print(match)
    for line in match:
        #print(line)
        can_rel.append(line)
print(can_rel)
        #print(re.findall('\".*\"', df['Body']))

['the first modern-day research on graviola was conducted in 1976 by the national cancer institute, though the plant has been under investigation since the 1940s.', 'their findings reported that the leaves of the graviola plant were effective in destroying malignant cancer cells.', 'a korean study found that graviola killed colon cancer cells better than a chemotherapy drug called adriamycin.', 'this means that there would likely be no hair loss or nausea as side effects from using graviola as a treatment for cancers.']


In [0]:

####### This code is for finding the score of each of the sentence from previous part by applying TextRank

import networkx as nx
import numpy as np
 
from nltk.tokenize.punkt import PunktSentenceTokenizer
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
score_textrank=[] 
sentences=can_rel
def textrank(can_rel):
    
    bow_matrix = CountVectorizer().fit_transform(sentences)
    normalized = TfidfTransformer().fit_transform(bow_matrix)
 
    similarity_graph = normalized * normalized.T
 
    nx_graph = nx.from_scipy_sparse_matrix(similarity_graph)
    scores = nx.pagerank(nx_graph)
    for i in range(len(scores)):
      #print(scores[i])
      score_textrank.append(scores[i])
    #print(((scores[i],s) for i,s in enumerate(sentences)))
    x= list((scores[i],s) for i,s in enumerate(sentences))
    print(x)
    return (((scores[i],s) for i,s in enumerate(sentences)))
                  #reverse=True)
out=textrank(can_rel)
print(out)
score_textrank_sorted=sorted(score_textrank, reverse=True)
print(score_textrank_sorted)

[(0.2525267693088695, 'the first modern-day research on graviola was conducted in 1976 by the national cancer institute, though the plant has been under investigation since the 1940s.'), (0.27218812841871776, 'their findings reported that the leaves of the graviola plant were effective in destroying malignant cancer cells.'), (0.23893664841881274, 'a korean study found that graviola killed colon cancer cells better than a chemotherapy drug called adriamycin.'), (0.23634845385359998, 'this means that there would likely be no hair loss or nausea as side effects from using graviola as a treatment for cancers.')]
<generator object textrank.<locals>.<genexpr> at 0x7f8732f895c8>
[0.27218812841871776, 0.2525267693088695, 0.23893664841881274, 0.23634845385359998]


In [0]:
######## This code gives the cosine score of each sentence by using sentence BERT


from sentence_transformers import SentenceTransformer
import scipy.spatial
import gensim
import string
import sys
#sys.stdout = open('C:/Users/Pritam/Desktop/Scores/sentence_sim_scores.txt', 'w', encoding="utf-8")
#from nltk.corpus import stopwords
#stop_words = set(stopwords.words('english'))
#sentence_list=[]
embedder = SentenceTransformer('roberta-large-nli-stsb-mean-tokens')

from nltk import sent_tokenize
corpus = can_rel
#print(sentences)
print(can_rel)
corpus_embeddings = embedder.encode(corpus)

# Query sentences:
from nltk import sent_tokenize
queries = sent_tokenize(sentence_main)
print(queries)

query_embeddings = embedder.encode(queries)

# Find the closest  sentences of the corpus for each query sentence based on cosine similarity
closest_n = 10
score_list=[]
for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]
    print(distances)
    print(range(len(distances)))
    results = zip(range(len(distances)), distances)
    x=list(results)
    print(x)
    #results = sorted(results, key=lambda x: x[1])
    #print(results)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTopmost similar sentences in corpus:")

    for idx, distance in x[0:closest_n]:
    	score=1-distance
      
    	print(corpus[idx].strip(), "(Score: %.4f)" % (score))
    	score_list.append(score)
print(score_list)
#from statistics import mean
#avg=mean(score_list)
#print("The average of the scores is: %.4f" % avg)
#sys.stdout.close()



['the first modern-day research on graviola was conducted in 1976 by the national cancer institute, though the plant has been under investigation since the 1940s.', 'their findings reported that the leaves of the graviola plant were effective in destroying malignant cancer cells.', 'a korean study found that graviola killed colon cancer cells better than a chemotherapy drug called adriamycin.', 'this means that there would likely be no hair loss or nausea as side effects from using graviola as a treatment for cancers.']
['graviola shows promise in cancer cures and arthritis.']
[0.58907033 0.38560702 0.38906662 0.44528538]
range(0, 4)
[(0, 0.5890703260663622), (1, 0.38560702269207614), (2, 0.3890666247090807), (3, 0.4452853761445118)]




Query: graviola shows promise in cancer cures and arthritis.

Topmost similar sentences in corpus:
the first modern-day research on graviola was conducted in 1976 by the national cancer institute, though the plant has been under investigation since the

In [0]:
######### The final code adds both the scores to get the final score according to which the highest score is given the most preference
import math
sum_list=[]     ########## list to hold the final sum of scores from TextRank and cosine score
for sc, lst in zip(score_textrank, score_list):  ########## taking the scores from both the lists 
  
  sig=(1 / (1 + math.e ** -(sc+lst)))   ########### using a sigmoid function to normalize the data so that it's between 0 and 1 everytime
  sum_list.append(sig)    ########## adding all the above results in the list
#print(sum_list)
#print(range(len(sum_list)))
result_final=zip(range(len(sum_list)),sum_list)      ####### adding the indexes to be used later
result_final=sorted(result_final, key=lambda x: x[1],reverse=True)   ########## sorting in descending order 
for query, query_embedding in zip(queries, query_embeddings):
    

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTopmost similar sentences in corpus:")

    for idx, sum_list in result_final[0:closest_n]:
    	
      
    	print(corpus[idx].strip(), "(Score: %.4f)" % (sum_list))    ########## the final list of sentences in descending order with the normalized scores
    	#score_list.append(score)

[0.6600364025318065, 0.7081841292560026, 0.7005398762700901, 0.6880595483895238]
range(0, 4)




Query: graviola shows promise in cancer cures and arthritis.

Topmost similar sentences in corpus:
their findings reported that the leaves of the graviola plant were effective in destroying malignant cancer cells. (Score: 0.7082)
a korean study found that graviola killed colon cancer cells better than a chemotherapy drug called adriamycin. (Score: 0.7005)
this means that there would likely be no hair loss or nausea as side effects from using graviola as a treatment for cancers. (Score: 0.6881)
the first modern-day research on graviola was conducted in 1976 by the national cancer institute, though the plant has been under investigation since the 1940s. (Score: 0.6600)


In [0]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
!pip install -U sentence-transformers

     |████████████████████████████████| 61kB 2.4MB/s 
     |████████████████████████████████| 450kB 10.1MB/s 
     |████████████████████████████████| 1.0MB 49.9MB/s 
     |████████████████████████████████| 870kB 54.5MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.5.1-cp36-none-any.whl size=67076 sha256=438867e7f74e2b76c5bf9f2dfa0c8acab23b8063e43a61c072099f679a6b28a0
  Stored in directory: /root/.cache/pip/wheels/22/ca/b4/7ca542b411730a8840f8e090df2ddacffa1c4dd9f209684c19
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=1935f6eca498115ff404b8866b8633d7986c89bb4c6568cda23fdb39c23fb0a4
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sentence-transformers sacremoses
